In [294]:
import pandas as pd
import numpy as np

In [296]:
df = pd.read_csv(r'C:\Users\GabrielCielo\Desktop\consolatio\relatorios\bling\estoque16jan2025.csv',delimiter=";")
df_saidas = pd.read_csv(r'C:\Users\GabrielCielo\Desktop\consolatio\relatorios\bling\saidasestoquejaneiro2025.csv',delimiter=";")

In [298]:
def pipeline_estoqueDB(df):
    """
    Passa um arquivo CSV contendo informações sobre estoque da Bling 
    por um pipeline de limpeza para que os dados possam ser visualizados no Looker
    """
    
    # Passo 1: Renomear a coluna 'Produto' para 'Informações'
    colunas_renomeadas = {'Produto':'Informacoes','Quantidade':'Estoque'}
    df.rename(columns=colunas_renomeadas, inplace=True)  

    # Passo 2: Separar a coluna 'Informações' em 'Produto', 'Cor' e 'Tamanho'
    df[['Produto','Cor','Tamanho']] = df['Informacoes'].str.extract(
    r'^(.*?) Cor:(.*?);Tamanho:([^;]+)') 

     # Passo 2: Deletar as colunas que não estão na lista
    colunas_desejadas = ['Código', 'Produto', 'Cor', 'Tamanho', 'Estoque']
    df = df[colunas_desejadas] 

    # Passo 4: Alterar a coluna 'Estoque' removendo a vírgula, os zeros e transformando em integer
    df.loc[:,('Estoque')] = df['Estoque'].fillna('0,00')  # Substitui NaN por '0,00'
    df.loc[:,('Estoque')] = df['Estoque'].str.split(',').str[0].astype(int)

    # Passo 5: Filtrar o dataframe para manter apenas os produtos na lista fornecida
    produtos = ['Camisa X-Tretch', 'Camisa Ultra-Stretch', 'Camisa Polo Ultra', 'Calça X-Tretch',
            'Camisa X-Tretch Manga Curta', 'Kit Cuecas 5', 'Kit Cuecas 10',
            'Necessaire', 'Calça Neo','Camiseta TecModal']
    df = df[df['Produto'].isin(produtos)]
    return df

In [300]:
def pipeline_saidas_de_estoque(df):
    """
    Passa um arquivo CSV contendo informações sobre entrada e saída de estoque da Bling 
    por um pipeline de limpeza para que os dados possam ser visualizados no Looker
    """

    df.drop(df.index[-1],inplace=True)
    
    # Passo 1: Renomear a coluna 'Produto' para 'Informações'
    colunas_renomeadas = {'Saldo antes de 01/01/2025':'Estoque Inicial',
        ' Valor Entradas *':'Valor Entradas',' Valor Saídas **':'Valor Saídas',
        ' Saldo em 31/01/2025':'Estoque'}
    df.rename(columns=colunas_renomeadas, inplace=True)  

    # Passo 2: Separar a coluna 'Informações' em 'Produto', 'Cor' e 'Tamanho'
    df[['Produto','Cor','Tamanho']] = df['Produto'].str.extract(r'^(.*?) Cor:(.*?);Tamanho:([^;]+)') 

     # Passo 2: Deletar as colunas que não estão na lista
    colunas_desejadas = ['Código', 'Produto', 'Cor', 'Tamanho', 
                         'Estoque Inicial', 'Entradas','Valor Entradas',
                         'Saídas','Valor Saídas','Estoque']
    df = df[colunas_desejadas] 

    # Passo 4: Alterar a coluna 'Estoque' removendo a vírgula, os zeros e transformando em integer
    colunas_para_manipular = ['Estoque Inicial','Entradas','Saídas','Estoque']
    df.loc[:,colunas_para_manipular] = df[colunas_para_manipular].apply(
        lambda x: x.fillna('0,00').str.split(',').str[0].astype(int) # Substitui NaN por '0,00'
    )
  
    # Passo 5: Filtrar o dataframe para manter apenas os produtos na lista fornecida
    produtos = ['Camisa X-Tretch', 'Camisa Ultra-Stretch', 'Camisa Polo Ultra', 'Calça X-Tretch',
            'Camisa X-Tretch Manga Curta', 'Kit Cuecas 5', 'Kit Cuecas 10',
            'Necessaire', 'Calça Neo','Camiseta TecModal']
    df = df[df['Produto'].isin(produtos)]
    return df

In [302]:
df_saidas = pipeline_saidas_de_estoque(df_saidas)
df_saidas.head()


,Código,Produto,Cor,Tamanho,Estoque Inicial,Entradas,Valor Entradas,Saídas,Valor Saídas,Estoque
0,CLNE-CAQU-040,Calça Neo,Caqui,40,0,0,"0,00",1,"349,90",-1
1,CLNE-CAQU-042,Calça Neo,Caqui,42,-1,0,"0,00",2,"699,80",-3
2,CLNE-CAQU-046,Calça Neo,Caqui,46,0,0,"0,00",1,"349,90",-1
3,CLNE-PRET-040,Calça Neo,Preto,40,3,0,"0,00",3,"1.049,70",0
4,CLNE-PRET-042,Calça Neo,Preto,42,6,0,"0,00",2,"699,80",4


In [304]:
df = pipeline_estoqueDB(df)
df.head()

,Código,Produto,Cor,Tamanho,Estoque
421,CLNE-CAQU-038,Calça Neo,Caqui,38,0
422,CLNE-CAQU-040,Calça Neo,Caqui,40,-1
423,CLNE-CAQU-042,Calça Neo,Caqui,42,-3
424,CLNE-CAQU-044,Calça Neo,Caqui,44,0
425,CLNE-CAQU-046,Calça Neo,Caqui,46,-1


In [312]:
merged = pd.merge(df,df_saidas[['Código','Estoque Inicial','Entradas','Valor Entradas','Saídas','Valor Saídas']], how='left',on='Código')
merged = merged.fillna(0)
merged.head()

C:\Users\GabrielCielo\AppData\Local\Temp\ipykernel_7416\3150518305.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged = merged.fillna(0)


,Código,Produto,Cor,Tamanho,Estoque,Estoque Inicial,Entradas,Valor Entradas,Saídas,Valor Saídas
0,CLNE-CAQU-038,Calça Neo,Caqui,38,0,0,0,0,0,0
1,CLNE-CAQU-040,Calça Neo,Caqui,40,-1,0,0,"0,00",1,"349,90"
2,CLNE-CAQU-042,Calça Neo,Caqui,42,-3,-1,0,"0,00",2,"699,80"
3,CLNE-CAQU-044,Calça Neo,Caqui,44,0,0,0,0,0,0
4,CLNE-CAQU-046,Calça Neo,Caqui,46,-1,0,0,"0,00",1,"349,90"


In [314]:
merged.to_csv(r'C:\Users\GabrielCielo\Desktop\consolatio\Databases\estoqueDB.csv',index=False)